In [1]:
import numpy as np
import sys
import scipy.stats
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../codes')

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
from VMSfunctions.Common import *
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *

# Mass Spec test

In [6]:
ps = load_obj('../models/peak_sampler_4_beers.p')
xcms_output = '../models/beer_ms1_peaks.csv.gz'
chromatograms = ChromatogramCreator(xcms_output)

c:\users\vinny\.virtualenvs\simulator-r22bo2kk\lib\site-packages\sklearn\base.py:253: UserWarning:

Trying to unpickle estimator KernelDensity from version 0.20.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.



0


../codes\VMSfunctions\Common.py:50: RuntimeWarning:

divide by zero encountered in double_scalars



5000
10000
15000
20000
25000
30000
35000
40000
45000


In [7]:
chemicals = ChemicalCreator(ps, chromatograms)

In [8]:
min_ms1_intensity = 2E5
min_rt = 3*60
max_rt = 21*60
dataset = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 100, 3, "Unknown","Empirical",None, False)

Warning ms_level > 3 not implemented properly yet. Uses ms_level = 2 information for now


In [9]:
scan_parameters = {
    'isolation_windows': [[(0, 1e3)]], # TODO: change to dictionary?
    'ms_level': 1
}    
sampled_chemicals = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 100, 2, "Unknown","Empirical",None, False)
mass_spec = IndependentMassSpectrometer(sampled_chemicals, scan_parameters)

In [27]:
scan_windows = {
    2: [(0, 20), (0, 1e3)],
}
mass_spec._get_all_mz_peaks(dataset[0], 810, 2, [[(0, 1e3)]])

[(67.4661379920823, 10948.055317032653), (87.62666577946767, 2264.746669422268), (119.09811334748623, 4058.560065347205), (71.53515090322713, 4257.21427928603), (122.73278830881634, 1847.0184666356133), (124.53176152723178, 2477.433642383482), (180.45913613188412, 2099.9177610604684), (145.29163704198518, 516.3099070801961), (134.44624543083398, 8534.871185006621), (77.77148014616239, 275.25250791252665), (275.58776247645955, 119.58833183514825), (70.5629803281249, 6829.1890317795), (63.81404260033734, 935.4721477016456), (111.77410613008331, 1261.8735595295525), (67.31226667655784, 658.2835827865454), (178.63490010417092, 4841.995627717692), (70.64959358219622, 24968.871895502383), (58.79629999155677, 1600.3393833798636), (580.2595507042162, 9500.823553345817), (89.23925218325577, 776.639449599284), (84.56579385478034, 1319.2936869834905), (65.89315289580847, 7760.316208749658), (57.46141889119433, 2222.9648266627514), (70.64397760845266, 4482.158784888476), (67.86195869508693, 18836.

In [20]:
print(dataset[0].children[0].parent_mass_prop)
print(dataset[0].children[0].prop_ms2_mass)
print(dataset[0].max_intensity)

0.42748731411043794
0.07434713245109743
3313961.883987893


In [26]:
344468.5121144637 * dataset[0].children[0].prop_ms2_mass * dataset[0].children[0].parent_mass_prop

10948.055317032653